# Web Scrape SQL Analysis

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

USER = os.getenv("PG_USER")
PASS = os.getenv("PG_PASSWORD")
HOST = os.getenv("PG_HOST")
DB   = os.getenv("PG_DB")

conn_str = f"postgresql+psycopg2://{USER}:{PASS}@{HOST}/{DB}"
engine   = create_engine(conn_str)

pd.set_option("display.max_rows", None)

## 1) Descriptive: Top Discussed F1 Drivers on Reddit

**Business Question:**  
Which F1 drivers generate the most discussion on r/formula1?  Fan chatter can reveal who’s on everyone’s mind, informing broadcast focus and sponsor value.


In [2]:
# Cell 2: Descriptive – Top 10 Most‑Mentioned Drivers on Reddit

sql_query = """
WITH 
drivers AS (
  SELECT DISTINCT driver
    FROM raw.qualifying_results_2023
),
mentions AS (
  SELECT
    d.driver,
    COUNT(*) AS mention_count
  FROM drivers d
  JOIN raw.web_scrape_formula1 w
    ON w.title ILIKE '%%' || d.driver || '%%'
  GROUP BY d.driver
)
SELECT
  driver,
  mention_count,
  RANK() OVER (ORDER BY mention_count DESC) AS mention_rank
FROM mentions
ORDER BY mention_count DESC;
"""

# Use keyword 'con=' so pandas knows this is the SQL connection,
# and nothing ends up in the wrong slot:
df_mentions = pd.read_sql(sql_query, engine)
df_mentions

,driver,mention_count,mention_rank
0,Norris,6,1
1,Verstappen,4,2
2,Piastri,4,2
3,Russell,3,4
4,Leclerc,3,4
5,Hamilton,2,6
6,Alonso,2,6
7,Albon,1,8
8,Sainz,1,8
9,Tsunoda,1,8


**Insights:**
- **Lando Norris leads discussion** with 6 mentions, well ahead of Verstappen and Piastri (4 each).  
- **Mid-field chatter drivers** like Russell and Leclerc (3 mentions) eclipse perennial champions Hamilton and Alonso (2 mentions), suggesting fresh fan interest.  
- **Lower-tier drivers** (Albon, Sainz, Tsunoda) appear only once, indicating limited fan focus.

**Recommendations:**
- **Feature Norris heavily** in post‑race social posts, highlight his storylines to capitalize on peak interest.  
- **Create content around rising stars** Piastri and Russell to sustain engagement—“Where are they now?” or “Top passes” montages.  
- **Engage Hamilton/Alonso fans** via targeted polls or “what surprised you most?” posts to reignite discussion.

**Predictions:**
- If Norris secures another podium this weekend, his mention count will likely rise by ≥50%, cementing margin over Verstappen.  
- A surprise podium by a mid‑field driver (e.g. Russell) will double their mention_count, shifting social media attention away from the usual front‑runners.

## 2) Diagnostic – Mentions vs. Race Outcome

**Business Question:**  
How do drivers’ race outcomes influence Reddit discussion volume? Understanding whether podiums, mid‑pack finishes, or DNFs spur more chatter can help GM Motorsports tailor its post‑race media strategy and engagement timing.


In [3]:
### Cell 3: Diagnostic – Do Top‑Mentioned Drivers Actually Finish Higher?

# Cell 3: Diagnostic – Mentions vs. Avg Finish Position
sql_query = """
WITH 
  -- reuse our mention counts from the descriptive query
  mentions AS (
    WITH drivers AS (
      SELECT DISTINCT driver
      FROM raw.qualifying_results_2023
    )
    SELECT
      d.driver,
      COUNT(*) AS mention_count
    FROM drivers d
    JOIN raw.web_scrape_formula1 w
      ON w.title ILIKE '%%' || d.driver || '%%'
    GROUP BY d.driver
  ),

  -- compute each driver’s average race finishing position
  avg_finish AS (
    SELECT
      driver,
      ROUND(AVG(finish::numeric), 2) AS avg_finish_pos,
      COUNT(*)                AS races_count
    FROM raw.race_results_2023
    GROUP BY driver
    HAVING COUNT(*) >= 5
  ),

  -- combine chatter with on‑track performance
  combined AS (
    SELECT
      m.driver,
      m.mention_count,
      f.avg_finish_pos,
      f.races_count
    FROM mentions m
    JOIN avg_finish f
      ON m.driver = f.driver
  )

SELECT
  driver,
  mention_count,
  avg_finish_pos,
  RANK() OVER (ORDER BY avg_finish_pos ASC) AS finish_rank
FROM combined
ORDER BY mention_count DESC
LIMIT 10;
"""

df_diag = pd.read_sql(sql_query, con=engine)
df_diag


,driver,mention_count,avg_finish_pos,finish_rank
0,Norris,6,13.2,8
1,Piastri,4,14.6,9
2,Verstappen,4,1.4,1
3,Leclerc,3,11.2,7
4,Russell,3,8.2,5
5,Alonso,2,3.2,2
6,Hamilton,2,4.8,3
7,Albon,1,14.8,10
8,Sainz,1,6.4,4
9,Tsunoda,1,10.6,6


**Insight:**  
- Top‑buzz Norris (mention_count = 6) surprisingly sits 8th in average finish (13.2), indicating fan excitement outpaces results.  
- Verstappen and Alonso convert chatter into performance: they rank 1st and 2nd in avg_finish_pos, matching high fan interest.  
- Mid‑tier buzz (Russell, Sainz) also deliver solid top‑5 average finishes, validating steady fan support.  
- Under‑mentioned Albon and Piastri struggle on track (finish_rank ≥ 9), highlighting missed storytelling opportunities.

**Recommendation:**  
- Align social campaigns with high‑buzz/ low‑finish gaps (e.g., “Why are fans so hyped on Norris despite mixed results?”) to drive engagement.  
- Promote data‑backed praise for consistent over‑achievers with moderate buzz (Russell, Sainz) to build broader fan narratives.  
- Spotlight low‑buzz/high‑finish surprises to cultivate emerging fan interest and sponsor ROI (e.g., “Silent Star: Alonso’s quiet charge”).

**Prediction:**  
- If Norris improves his finish by 3 positions next race (to ≈10th), his mention_count will drop by ~20% as fan optimism tempers to realism.  
- A low‑buzz driver achieving a podium (finish_rank ≤ 3) will likely double their mention_count within 24 hrs of the race, driving fresh social spikes.  
- Consistent top‑5 finishes by a mid‑buzz driver (Russell/Sainz) over a 3‑race span will increase their mention_count by at least 30% as word‑of‑mouth spreads.